In [159]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')

import torch
import plotly.express as px
import src.nanda_plot
from src.nanda_plot import imshow_reversed, imshow
import pandas as pd
import numpy as np

from src.result_analyzer import ResultAnalyzer

## Generate the CSV file

In [204]:
result = ResultAnalyzer("gpt2_full_result.pt")

In [205]:
result.process_pos_attn_head_out()

,id,mean,std,t-test,p-value
0,L0H0,0.002512,0.007592,0.320331,0.753449
1,L0H1,0.004450,0.012541,0.343595,0.736257
2,L0H2,-0.000952,0.003635,-0.253613,0.803481
3,L0H3,-0.000060,0.007064,-0.008252,0.993532
4,L0H4,0.003325,0.012502,0.257492,0.800545
...,...,...,...,...,...
139,L11H7,0.002666,0.002683,0.962039,0.352353
140,L11H8,0.002068,0.002280,0.877898,0.394816
141,L11H9,0.001654,0.008557,0.187151,0.854227
142,L11H10,-0.013669,0.011912,-1.111054,0.285263


In [185]:
result.process_pos_attn_head_out()

,id,mean,std,t-test,p-value
0,L0H0,0.002512,0.007592,0.320331,0.753449
1,L0H1,0.004450,0.012541,0.343595,0.736257
2,L0H2,-0.000952,0.003635,-0.253613,0.803481
3,L0H3,-0.000060,0.007064,-0.008252,0.993532
4,L0H4,0.003325,0.012502,0.257492,0.800545
...,...,...,...,...,...
139,L11H7,0.002666,0.002683,0.962039,0.352353
140,L11H8,0.002068,0.002280,0.877898,0.394816
141,L11H9,0.001654,0.008557,0.187151,0.854227
142,L11H10,-0.013669,0.011912,-1.111054,0.285263


## Make plot

In [20]:
data["pos"].keys()

dict_keys(['attn_head_out', 'mlp_out', 'attn_out_by_pos', 'example_str_tokens'])

In [31]:
data["pos"]["attn_head_out"][0].keys()

'Honda NS500 is produced byhel Honda NS500 is produced by'

## Positive examples

In [27]:
imshow(
    data["pos"]["attn_head_out"][0]["mean"],
    yaxis="layer",
    xaxis="head",
    title="attn_head_out MEAN",
)
imshow(
    data["pos"]["attn_head_out"][0]["std"],
    yaxis="layer",
    xaxis="head",
    title="attn_head_out STD",
)
imshow(
    data["pos"]["attn_head_out"][0]["p-value"],
    yaxis="layer",
    xaxis="head",
    title="attn_head_out p-value",
)

In [36]:
from src.model import WrapHookedTransformer
model = WrapHookedTransformer.from_pretrained("gpt2")

pos_example = model.to_str_tokens(data["pos"]["example_str_tokens"][0])

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [39]:
imshow(
    data["pos"]["mlp_out"][0]["mean"],
   
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(pos_example)],
)
imshow(
    data["pos"]["mlp_out"][0]["std"],
   
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(pos_example)],
)
imshow(
    data["pos"]["mlp_out"][0]["p-value"],
   
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(pos_example)],
)

In [299]:
imshow(
    data["neg"]["patch_attn_head_out"]["mean"].mean(0),
    yaxis="Layer", 
    xaxis="Head", 
    title = "Attention heads"
)
imshow(
    data_var["neg"]["patch_attn_head_out"]["mean"].mean(0),
    yaxis="Layer", 
    xaxis="Head", 
    title = "Attention heads (MAD)",
    # zmax=1,
    # zmin=-1
)
imshow(
    data["neg"]["patch_per_block"]["mean"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)
imshow(
    data_var["neg"]["patch_per_block"]["mean"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block (MAD)",
    xaxis="Position",
    yaxis="Layer",
    # zmax= 6,
    # zmin=-6,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["neg"]["example_str_token"])],
)

### Subsample

In [294]:
imshow(
    data["neg"]["patch_per_block"]["mean"][0,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)

In [295]:
imshow(
    data["neg"]["patch_per_block"]["mean"][1,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)